# SIMULATING A BULK WATER SYSTEM USING CAR-PARRINELLO MOLECULAR DYNAMICS

<div style="text-align: center"><i>Alessandro Cuoghi - April 2023</i></div>

--- 

# *I* - INTRODUCTION
In studying the physics of complex systems, numerical simulations provide an essential tool to determine the values of quantities that are difficult to obtain through experimental techniques, as well as to verify theories or to examine how certain principles affect the dynamics of the system. From this perspective, studies performed via numerical simulations allow also to examine unphysical processes.

Classical molecular dynamics (MD) is a commonly used computational tool for simulating the properties of liquids, solids, and molecules. Each of the atoms or molecules in the simulation is treated as a point mass and Newton’s equations are integrated to compute their motion. In classical MD the forces are derived from empirical laws and the accuracy of the results strongly depends on the chosen model. This approach has enjoyed tremendous success in the treatment of systems ranging from simple liquids and solids to polymers and biological systems such as proteins and nucleic acids. Despite their success, force fields have a number of serious limitations, the major being that, first, these calculations convey no information about electronic properties and, second, force fields generally assume a pre-specified connectivity among the atoms and, therefore, suffer from an inability to describe chemical bond breaking and forming events. On the other hand, density functional theory (DFT) calculations have provided an accurate, albeit approximate, description of the chemical bond in a large variety of systems, altough they are very demanding computationally and it is only realistically possible to do small simulations for short times. This limits the application of DFT schemes to the study of very large and/or disordered systems and to the computation of interatomic forces for MD simulations.

# *II* - AB INITIO MOLECULAR DYNAMICS
The techniques of every MD scheme face with two differents problems:
- how to compute the trajectory by numerically integrating the classical equation of motion for every particle in the system.
- how to define a force field that correctly describes the physics of the system.
 
## *II.I* - INTEGRATING THE EQUATION OF MOTION. THE VERLET ALGORITHM
In order to evolve the position and the velocity of a particle and computen the trajectory, time is *discretized* in many steps of equal length $\Delta t$. Then, the position of every particle $I$ at a successive step $\vec R_I(t+\Delta t)$ depends on the position, velocity and force at the previous time step $t$

$$\vec R_I(t+\Delta t) = \vec R_I(t) + \vec v_I(t)\Delta t + \frac{\vec F_I(t)}{2m_I}\Delta t^2 + \mathcal{O}(\Delta t^3)$$

There exist many algorithms that allow to integrate Newton's equation of motion; one of the most used and the one that is implemented in the simulations for this report is the Verlet's algorithm [$^{[1]}$](#bib-ver) 

<a id='eq-ver'></a>
$$\vec R_I(t+\Delta t) = 2\vec R_I(t) - \vec R_I(t-\Delta t) + \frac{\vec F_I(t)}{2m_I}\Delta t^2 \tag{1}$$

for which is possible to demonstrate that has an error of $\mathcal{O}(\Delta t^4)$. As we want an efficient simulation that allows to compute quantities of interest with precise values, we want to introduce a numerical error that is as small as possible. Thus, it is crucial the choice of the timestep $\Delta t$: it should be very small so that to minimize the error of the integration procedure, but on the other hand, it should be big enough to allow the simulation to run for a given amount of time needed to simulate properly the physics we are interested in within the computational resources available. It turns out that for a Verlet scheme [(Eq. 1)](#eq-ver), a good choice for the timestep is to take the fastest characteristic time of the system and divide it by 30, $\Delta t \approx \tau/30$.

A general scheme for a MD simulation is composed of 4 steps:
1. *Initialize positions and velocities for every particle of the system*. The velocities are found straightforward by sampling the Boltzmann's distribution at a given temperature; the atomic positions must be carefully determined and chosen such that the system is in a relaxed configuration. Hence, most of the time it is a good idea to run a preliminary simulation with a classical force field to get a more or less thermalized initial state to start with. 

2. *Compute the forces and integrate the equation of motion*. As already discussed, this is done by means of the Verlet algorithm [(Eq. 1)](#eq-ver).

3. *Equilibrate the system*, i.e. allow it to reach thermal equilibrium. Altough it is really difficult to let the system staibilize at a *chosen target temperature*; this has to be with the fact that Newton's equation preserves the total energy of the system and thus the simulation is run in what is called the *microcanonical ensemble*. Temperature is an output of the system, not an input. In order to thermalize the system at a target temperature we need to move to the *canonical ensemble*. This is achieved by applying the Nosé–Hoover thermostat[$^{[2]}$](#bib-hoo), so that the system equilibrate at a target temperature. Care must be taken, since the thermostat changes the equation of motion; thus, in order to get meaningful results from the simulation, after the equilibration one must switch off the thermostat and go back to classical equation of motion. Hopefully, the temperature will then remain very close to the target one. 

4. *Compute the physical quantities of interest*. In MD the values of the physical quantities are extracted by performing averages over time and over the ensemble of particles. 

## *II.II* - COMPUTING THE FORCE FIELD. CAR-PARRINELLO MOLECULAR DYNAMICS
We recall from [(Eq. 1)](#eq-ver) that in order to compute the trajectory we need to evaluate at each timestep the forces acting on every particles. They are computed as the derivative of the total energy with respect to the coordinates,

<a id='eq-for'></a>
$$F_I = -\frac{\partial E(\{R\})}{\partial R_I} \tag{2}$$

In classical MD, the energy is computed as a sum of empirical potentials, that usually comprise for example a two-body pair potential (could be the Lennard-Jones potential) with a 3-body correction term and a mean-field term proportional to the density. The more the terms included, the more accurate the model is. 

**Ab initio Molecular Dynamics** consists of techniques that compute the force field *ab initio* starting from the electronic configurations around the nuclei. At every step, *Schroedinger equation* must be solved in order to find the electronic wavefunction and the energy of the system. Then, by means of the *Hellmann-Feynman theorem* we can compute its derivative in an efficient way to evaluate the forces [(Eq. 2)](#eq-for), so that we can evolve the ions coordinates. Nuclei are still considered as classical particles and obey Newton's equation, but in order to compute the force we need to solve quantum mechanical problem. *Born-Oppenheimer Molecular Dynamics* (BOMD) assumes that electrons stay in their ground state and that the electronic configuration only depends on the current atomic configurations (*adiabaticity assumption*), that is, we neglect every contributions to the dynamics that comes from "the history" of the electronic wavefunction.

Solving at each step the Schroedinger equation and then evolve the atomic positions is a huge and expensive computational task and hence ab-initio MD can simulate smaller systems for a reduced amount of time with respect to classical MD. However, since at each timestep the displacement of the nuclei is very small we can expect that the electronic wavefunction does not change that much every timestep and indeed we can take advantage of this fact to evolve the wavefunction. It is not necessary to solve the quantum mechanical probelm from scrtach every time. This is the fundamental assumption of *Car-Parrinello Molecular Dynamics* (CPMD)[$^{[3]}$](#bib-cpmd). We can calculate ”forces” over each point in the space of the wavefunctions and treat them as a classical fluid that is running after a potential that changes at each step, and propagate everything with the Verlet algorithm. It is sort of dynamical simulated annealing, as explained in the original paper. The "classical" Lagrangian used for "evolve" the wavefunction is

<a id='eq-cpla'></a>
$$\mathcal{L} = \sum_\nu \frac{\mu}{2}\int_\Omega d^3r \ |\dot\psi_\nu|^2 + \sum_I \frac{m_I}{2}\dot R_I^2 - E[\{\psi\}, \{R\}] + \sum_{ij}\Lambda_{ij}\left(\int_\Omega d^3r \ \psi_i^*\psi_j-\delta_{ij}\right) \tag{3}$$

where $\mu$ is the ”fake” mass of the electrons, $E$ is the energy functional and the last term represents the orthonormality constraints that keeps the electronic orbitals orthogonal to each other, $\Lambda_{ij}$ being the Lagrangian multipliers. This is in the end a classical Lagrangian, from which we can derive classical equation of motion using Euler-Lagrange equation

<a id='eq-eul'></a>
$$\frac{\partial \mathcal{L}}{\partial x_i} = \frac{d}{dt}\frac{\partial \mathcal{L}}{\partial \dot x_i} \tag{4}$$

Note here that the coordinates that we are using to derive the dynamics are the electron wavefuntions coordinates (one coordinate for the value and one for the time derivative for each point in the discretized space) and the atomic coordinates (usual velocity and positions), as well as the Lagrangian multipliers. The great improvement introduced by CPMD is now clear: the electronic wavefunctions and the atomic coordinates are evolved **simultaneously**, obeying the equations of motion

<a id='eq-cpmd'></a>
$$
\begin{align}
\mu \ddot\psi_\nu(r,t) &= -\frac{\delta E}{\delta \psi^*_\nu} + \sum_{j}\Lambda_{\nu j}\psi_j \tag{5.a}\\
M_I \ddot {\vec R_I} &= -\frac{\partial E}{\partial R_I} \tag{5.b}
\end{align}
$$

Both will be integrated using Verlet scheme [(Eq. 1)](#eq-ver). The first equation of motion [(Eq. 5.a)](#eq-cpmd) tells us that the electronic wavefunction oscillates around the ground state, following "adiabatically" the true minimum of the energy, so that BOMD and CPMD represents "on average" the same dynamics.

# *III* - CODE

In order to run a simulation we will usage the `cp.x` package from Quantum Espresso[$^{[5,6]}$](#bib-qe1). We have to specify the parameters of the computation in an input file, which is organized into several namelists, followed by other fields (“cards”) introduced by keywords. The namelists are:
- `&CONTROL`: general variables controlling the run, for example the title, the type of calculation, the path to the needed directories, as well as the number of steps and the timestep of the run. 
- `&SYSTEM`: structural information on the system under investigation. The system comprises $32$ water molecules placed in a cubic box of length $10\ Å$ so that the density is around $0.958\ g/cm^3$. 
```
 ibrav=1, celldm(1)=10.0    ! cubic lattice of side celldm(1) in Bohr radius
 nat = 96,                  ! number of atoms
 ntyp = 2,                  ! type of atoms
 ecutwfc = 50.0,            ! kinetic energy cutoff (Ry) for wavefunctions
```
- `&ELECTRONS`: set the electronic variables and the electron dynamics.
- `&IONS`: set ionic variables and dynamics.

After the namelists, you have several fields (“cards”) introduced by keywords with self-explanatory names:
- `ATOMIC_SPECIES`: it is specified the pesudopotentials used for each atom type
```
 O  16.0d0   O_ONCV_PBE-1.2.upf
 H  1.008d0 H_ONCV_PBE-1.2.upf
```
- `ATOMIC_POSITIONS`: the coordinates of the atoms. The initial disposition of the water molecules inside the simulation box is generated by the package *packmol*[$^{[4]}$](#bib-pack).

In [1]:
from Code.gen_box import *
show_system('packmol/water_box.xyz', L)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## III.I REACHING THE ELECTRONIC GROUND STATE - BOMD

It is not a good idea to start immediately with a CPMD simulation because if you start assigning random velocities to the atoms, you can have some sharp changes in the atomic velocities because they could be not consistent with the current atomic positions, or because two atoms could be too near to each other; this will reflect in little kinks to the nuclei and by consequence little kinks to the electronic degrees of freedom, that will cause a little more kinetic energy to the electronic system (that we want as cold as possible) and a little rise in the electronic temperature. Thus the first run, when starting from scratch, is always an electronic minimization, with fixed ions and cell, to bring the electronic system on the ground state relative to the starting atomic configuration, and we are going to do it by means of the *conjugated gradient* method.

In [2]:
from IPython import display

In [5]:
display.Code(filename='Input/cp.water8.1-bomd.in', language='f90')

&control
  title = 'Water 32 molecules',
  calculation = 'cp',
  restart_mode = 'from_scratch',
  ndw = 50,                                      ! number directory write
  nstep = 25,                                    ! number of steps of the run
  iprint = 10,                                   ! frequency for printing the quantities
  isave = 1000,                                  ! frequency for wrting a restart file
  tprnfor = .TRUE.,                              ! compute the forces
  dt = 5.0d0,                                    ! timestep in Hartree atomic units
  prefix = 'H2O',
  pseudo_dir='/home/maglab1/ACuoghi/QE/pseudo/', ! pseudopotential directory
  outdir='/home/maglab1/ACuoghi/QE/H2O/',        ! output directory
/

&system
  ibrav=1, celldm(1)=10.0                        ! cubic lattice of side celldm(1) in Bohr radius
  nat = 96,                                      ! number of atoms
  ntyp = 2,                                      ! type of atoms
  ecutwfc = 50.0,                                ! kinetic energy cutoff (Ry) for wavefunctions
/

&electrons
  electron_dynamics = 'cg',
/
 
&ions
  ion_dynamics = 'verlet',
  ion_velocities = 'random',                     ! sampling the Boltzmann's distribution at a given 
  tempw=600.d0                                   ! temperature 
/
 
ATOMIC_SPECIES
 O  16.0d0   O_ONCV_PBE-1.2.upf
 H  1.0079d0 H_ONCV_PBE-1.2.upf

ATOMIC_POSITIONS (angstrom)
 O            9.498370        3.721840        2.768978
 H            9.945279        3.356063        3.585357
 H            8.536024        3.472563        2.877363

# *IV* - RESULTS


# *V* - CONCLUSIONS

---
# APPENDIX

# BIBLIOGRAPHY
1. *L. Verlet*, [Phys. Rev. **159**, 98 (1967)](https://doi.org/10.1103/PhysRev.159.98).<a id='bib-ver'></a>
2. *W. G. Hoover*, [Phys. Rev. A **31**, 1695 (1985)](https://doi.org/10.1103/PhysRevA.31.1695).<a id='bib-hoo'></a>
3. *R. Car and M. Parrinello*, [Phys. Rev. Lett. **55**, 2471 (1985)](https://doi.org/10.1103/PhysRevLett.55.2471).<a id='bib-cpmd'></a>
4. *L. Martínez, R. Andrade, E. G. Birgin and J. M. Martínez*, [J. Comput. Chem. 30(13):2157-2164 (2009)](https://doi.org/10.1002/jcc.21224).<a id='bib-pack'></a>
5. *P. Giannozzi et al.*, [J. Phys.: Condens. Matter 21, 395502 (2009)](https://doi.org/10.1088/0953-8984/21/39/395502)<a id='bib-qe1'></a>
6. *P. Giannozzi et al.*, [J. Phys.: Condens. Matter 29, 465901 (2017)](https://doi.org/10.1088/1361-648X/aa8f79)<a id='bib-qe2'></a>

In [ ]:
import os
import time
import numpy as np

In [ ]:
run_time = np.zeros((8,8))

for i in range(8):
    n_mpi = i+1
    for j in range(8):
        n_omp = j+1
        if n_mpi*n_omp<=8:
            for k in range(3):
                t0 = time.time()
                cmd_mpirun = f'mpirun -n {n_mpi} -genv OMP_NUM_THREADS={n_omp} -genv I_MPI_PIN_DOMAIN=omp '
                os.system(cmd_mpirun + 'cp.x -in Day-8/cp.water8.1-bomd.in')
                t = time.time()-t0
            
                run_time[i,j] += t/3
        else:
            run_time[i,j] = np.nan
            
run_time

In [ ]:
minim = 100
for i in range(8):
    for j in range(8):
        if run_time[i,j] < minim:
            minim = run_time[i,j]
            x, y = i, j
run_time, x, y, run_time[x,y]